# Grama: Fitting Multivariate Distributions

*Purpose*: As we've seen through studying a lot of datasets, many physical systems exhibit variability and other forms of uncertainty. Thus, it is beneficial to be able to *model* uncertainty using distributions. In the previous part of this exercise, we learned how to fit a distribution for a *single* quantity. Now we'll learn how to deal with *multiple* related quantities.

In the final exercise `e-grama08-duu` we'll see how to use a distribution model to do useful engineering work.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

For this exercise, we'll study a dataset of observations on die cast aluminum parts.


In [ ]:
from grama.data import df_shewhart


# Dependency in the wild

We've seen lots of examples of dependency in datasets so far! Any case where we see "patterns" in a scatterplot are examples of some relationship (or dependency) between variables.

As a concrete example, let's take a look at the measured `density` and `hardness` values from the cast aluminum dataset:


In [ ]:
## NOTE: No need to edit
(
    df_shewhart
    >> gr.ggplot(gr.aes("density", "hardness"))
    + gr.geom_point()
)

We can see a positive correlation between the two variables. If we wanted to model the `density` and `hardness` simultaneously, we would need to respect this dependency.

A normal distribution does a *reasonable* job representing both `density` and `hardness` separately.


In [ ]:
## NOTE: No need to edit
(
    df_shewhart
    >> gr.tf_mutate(
        q_density=gr.qqvals(DF.density, "norm"),
        q_hardness=gr.qqvals(DF.hardness, "norm"),
    )
    >> gr.tf_rename(
        v_density="density",
        v_hardness="hardness",
    )
    >> gr.tf_pivot_longer(
        columns=["q_density", "q_hardness", "v_density", "v_hardness"],
        names_to=[".value", "var"],
        names_sep="_",
    )
    
    >> gr.ggplot(gr.aes("q", "v"))
    + gr.geom_abline(intercept=0, slope=1, linetype="dashed")
    + gr.geom_point()
    + gr.facet_wrap("var", scales="free")
    + gr.labs(x="Reference Quantile", y="Observed Quantile")
)

However, if we fit a normal for `hardness` and `density` separately, *and ignore the dependency*, we will end up with a model that does not respect the correlation we saw above:


In [ ]:
## NOTE: No need to edit
# Build a model 
md_independence = (
    gr.Model("Independent Properties")
    >> gr.cp_marginals(
        density=gr.marg_fit("norm", df_shewhart.density),
        hardness=gr.marg_fit("norm", df_shewhart.hardness),
    )
    ## NOTE: This line is where things go wrong!
    # We can't just assume variables are independent
    # unless we have a good reason to do so.
    >> gr.cp_copula_independence()
)

# Draw simulated observations
(
    md_independence
    >> gr.ev_sample(n=1e3, df_det="nom", skip=True)
    >> gr.tf_mutate(source="Simulated")
    >> gr.tf_bind_rows(
        df_shewhart
        >> gr.tf_mutate(source="Experimental")
    )
    
    >> gr.ggplot(gr.aes("density", "hardness"))
    + gr.geom_point(gr.aes(color="source"))
    + gr.theme_minimal()
)

Notice that this model does not respect the correlation we observe in the data. How can we fix this?


# Marginal-Copula Approach

It turns out there is a drop-in solution for representing dependency; a [*copula*](https://en.wikipedia.org/wiki/Copula_(probability_theory)) is a mathematical tool to represent dependency between two (or more) random variables. We can fit a gaussian copula to represent the correlation we see between `hardness` and `density`.


In [ ]:
## NOTE: No need to edit
# Build a model 
md_copula = (
    gr.Model("Independent Properties")
    >> gr.cp_marginals(
        density=gr.marg_fit("norm", df_shewhart.density),
        hardness=gr.marg_fit("norm", df_shewhart.hardness),
    )
    ## KEY DIFFERENCE: Fit a gaussian copula
    >> gr.cp_copula_gaussian(df_data=df_shewhart)
)

# Draw simulated observations
(
    md_copula
    >> gr.ev_sample(n=1e3, df_det="nom", skip=True)
    >> gr.tf_mutate(source="Simulated")
    >> gr.tf_bind_rows(
        df_shewhart
        >> gr.tf_mutate(source="Experimental")
    )
    
    >> gr.ggplot(gr.aes("density", "hardness"))
    + gr.geom_point(gr.aes(color="source"))
    + gr.theme_minimal()
)

Note that this version of the model respects the correlation in the data!


## Steps

When fitting a density for *multiple* uncertainties, we should follow an extended process:

1. Fit a marginal for each uncertain quantity
    - Follow the process from `e-grama06-fit-univar`; this should include checking for statistical control!
2. Fit a copula to relate the uncertain quantities
3. Assess the model


# Case Study: Circuit Performance

To illustrate the full distribution-fitting process, we'll work through a case study of circuit performance.


In [ ]:
## NOTE: No need to edit
from grama.models import make_prlc_rand
md_circuit = make_prlc_rand()
md_circuit

This circuit model has variability in its performance due to manufacturing variability. If we inspect its two outputs `Q` and `omega0`, we can see that they are quite variable, and exhibit a strong negative correlation.


In [ ]:
## NOTE: No need to edit
df_circuit = (
    md_circuit
    >> gr.ev_sample(n=1e4, df_det="nom", seed=101)
)

(
    df_circuit
    >> gr.ggplot(gr.aes("Q", "omega0"))
    + gr.geom_bin2d()
)

## Marginals

First, let's fit distributions for the two quantities `Q` and `omega0` separately.


### __q1__ Check for statistical control

Check for statistical control of the output `omega0`. Use a large batch size. Answer the questions under *observations* below.


In [ ]:
## TASK: Make a control chart of batch size n_batch = 10
(
    df_circuit

)

*Observations*

- What batch size did you choose?
  - (Your response here)
- Does this process seem to be under statistical control?
  - (Your response here)
- Remind me: Is this quantity random? How do you know?
  - (Your response here)


### __q2__ Fit a marginal for `omega0`

Select a reasonable distribution for `omega0` and fit it using the data `df_circuit.omega0`. Answer the questions under *observations* below.


In [ ]:
## TASK: Fit a marginal for `omega0`
mg_omega0 = None


## NOTE: Use the following to help check your work
(
    df_circuit
    >> gr.tf_mutate(q=gr.qqvals(DF.omega0, marg=mg_omega0))
    >> gr.ggplot(gr.aes("q", "omega0"))
    + gr.geom_abline(intercept=0, slope=1, linetype="dashed")
    + gr.geom_point()
)

*Observations*

- How well does your distribution fit the data?
  - (Your response here)


### __q3__ Fit a marginal for `Q`

Select a reasonable distribution for `Q` and fit it using the data `df_circuit.Q`. Answer the questions under *observations* below.


In [ ]:
## TASK: Fit a marginal for `Q`
mg_Q = None


## HINT: Use the assessment techniques discussed in 
# the previous exercise


*Observations*

- How well does your distribution fit the data?
  - (Your response here)


## Copula

Now that we've fit marginals for `Q` and `omega0`, we can work on representing their dependency.

The following code fits a model that neglects the dependency between `Q` and `omega0`. You will fit a gaussian copula and compare against this baseline model.


In [ ]:
## NOTE: No need to edit
md_out_independence = (
    gr.Model("Circuit Output: Independence")
    >> gr.cp_marginals(
        omega0=mg_omega0,
        Q=mg_Q,
    )
    >> gr.cp_copula_independence()
)

md_out_independence 


### __q4__ Fit a gaussian copula

Add a gaussian copula to the model `md_out_copula`.

*Hint*: The code above demonstrates how to add a gaussian copula to a grama model.


In [ ]:
## TASK: Fit a gaussian copula model
md_out_copula = (
    gr.Model("Circuit Output: Copula")
    >> gr.cp_marginals(
        omega0=mg_omega0,
        Q=mg_Q,
    )

)

## NOTE: Do not edit; use this to check your work
assert \
    isinstance(md_out_copula.density.copula, gr.CopulaGaussian), \
    "md_out_copula must have a gaussian copula"

md_out_copula 


### __q5__ Compare the models

Use the following code to compare the multivariate models; answer the questions under *observations* below.


In [ ]:
## NOTE: No need to edit
(
    df_circuit
    >> gr.tf_mutate(source="True")
    >> gr.tf_bind_rows(
        md_out_independence
        >> gr.ev_sample(n=1e4, df_det="nom", skip=True)
        >> gr.tf_mutate(source="Independence")
    )
    >> gr.tf_bind_rows(
        md_out_copula
        >> gr.ev_sample(n=1e4, df_det="nom", skip=True)
        >> gr.tf_mutate(source="Copula")
    )
    
    >> gr.ggplot(gr.aes("Q", "omega0"))
    + gr.geom_bin2d()
    + gr.facet_wrap("source")
)

*Observations*

- How well does the independence model represent the true data?
  - (Your response here)
- What aspects does the copula model get *correct*?
  - (Your response here)
- What aspects does the copula model get *incorrect*?
  - (Your response here)
